# Exercise 2: Regression

# Import Required Libraries
using CSV
using DataFrames
using Statistics
using StatsBase
using Plots

In [19]:
using CSV
using DataFrames
using Statistics
using StatsBase
using Plots
using GLM

## Load Dataset

In [2]:
# Load Dataset
println("Cargando dataset...")
df = CSV.read("bottle.csv", DataFrame)

Cargando dataset...


Row,Cst_Cnt,Btl_Cnt,Sta_ID,Depth_ID,Depthm,T_degC,Salnty,O2ml_L,STheta,O2Sat,Oxy_µmol/Kg,BtlNum,RecInd,T_prec,T_qual,S_prec,S_qual,P_qual,O_qual,SThtaq,O2Satq,ChlorA,Chlqua,Phaeop,Phaqua,PO4uM,PO4q,SiO3uM,SiO3qu,NO2uM,NO2q,NO3uM,NO3q,NH3uM,NH3q,C14As1,C14A1p,C14A1q,C14As2,C14A2p,C14A2q,DarkAs,DarkAp,DarkAq,MeanAs,MeanAp,MeanAq,IncTim,LightP,R_Depth,R_TEMP,R_POTEMP,R_SALINITY,R_SIGMA,R_SVA,R_DYNHT,R_O2,R_O2Sat,R_SIO3,R_PO4,R_NO3,R_NO2,R_NH4,R_CHLA,R_PHAEO,R_PRES,R_SAMP,DIC1,DIC2,TA1,TA2,pH2,pH1,DIC Quality Comment
,Int64,Int64,String15,String,Int64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Int64?,Int64,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Float64?,Int64?,Float64?,Int64?,Float64?,Int64?,Float64?,Int64?,Float64?,Int64?,Float64?,Int64?,Float64?,Int64?,Float64?,Int64?,Int64?,Float64?,Int64?,Int64?,Float64?,Int64?,Int64?,Float64?,Int64?,Int64?,String31?,Float64?,Float64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Int64,Int64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,String?
1,1,1,054.0 056.0,19-4903CR-HY-060-0930-05400560-0000A-3,0,10.5,33.44,missing,25.649,missing,missing,missing,3,1,missing,2,missing,9,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,missing,9,missing,missing,9,missing,missing,9,missing,missing,9,missing,missing,0.0,10.5,10.5,33.44,25.64,233.0,0.0,missing,missing,missing,missing,missing,missing,missing,missing,missing,0,missing,missing,missing,missing,missing,missing,missing,missing
2,1,2,054.0 056.0,19-4903CR-HY-060-0930-05400560-0008A-3,8,10.46,33.44,missing,25.656,missing,missing,missing,3,2,missing,2,missing,9,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,missing,9,missing,missing,9,missing,missing,9,missing,missing,9,missing,missing,8.0,10.46,10.46,33.44,25.65,232.5,0.01,missing,missing,missing,missing,missing,missing,missing,missing,missing,8,missing,missing,missing,missing,missing,missing,missing,missing
3,1,3,054.0 056.0,19-4903CR-HY-060-0930-05400560-0010A-7,10,10.46,33.437,missing,25.654,missing,missing,missing,7,2,missing,3,missing,9,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,missing,9,missing,missing,9,missing,missing,9,missing,missing,9,missing,missing,10.0,10.46,10.46,33.437,25.65,232.8,0.02,missing,missing,missing,missing,missing,missing,missing,missing,missing,10,missing,missing,missing,missing,missing,missing,missing,missing
4,1,4,054.0 056.0,19-4903CR-HY-060-0930-05400560-0019A-3,19,10.45,33.42,missing,25.643,missing,missing,missing,3,2,missing,2,missing,9,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,missing,9,missing,missing,9,missing,missing,9,missing,missing,9,missing,missing,19.0,10.45,10.45,33.42,25.64,234.1,0.04,missing,missing,missing,missing,missing,missing,missing,missing,missing,19,missing,missing,missing,missing,missing,missing,missing,missing
5,1,5,054.0 056.0,19-4903CR-HY-060-0930-05400560-0020A-7,20,10.45,33.421,missing,25.643,missing,missing,missing,7,2,missing,3,missing,9,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,missing,9,missing,missing,9,missing,missing,9,missing,missing,9,missing,missing,20.0,10.45,10.45,33.421,25.64,234.0,0.04,missing,missing,missing,missing,missing,missing,missing,missing,missing,20,missing,missing,missing,missing,missing,missing,missing,missing
6,1,6,054.0 056.0,19-4903CR-HY-060-0930-05400560-0030A-7,30,10.45,33.431,missing,25.651,missing,missing,missing,7,2,missing,3,missing,9,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,9,missing,missing,9,missing,missing,9,missing,missing,9,missing,missing,9,missing,missing,30.0,10.45,10.45,33.431,25.65,233.5,0.07,missing,missing,missing,missing,missing,missing,missing,missing,missing,30,missing,missing,missing,missing,missing,missin

## Remove Columns with High Missing Values

In [3]:
# Función para calcular el porcentaje de valores faltantes por columna
# @param df: DataFrame
# @return: Diccionario con el nombre de la columna como clave y el porcentaje de valores faltantes como valor
function porcentaje_faltantes(df::DataFrame)
    porcentajes_faltantes = Dict()
    for col in names(df)
        conteo_faltantes = count(ismissing, df[:, col])
        porcentajes_faltantes[col] = (conteo_faltantes / nrow(df)) * 100
    end
    return porcentajes_faltantes
end

porcentaje_faltantes (generic function with 1 method)

In [4]:
# Función para eliminar columnas con un porcentaje de valores faltantes mayor al umbral
# @param df: DataFrame
# @param umbral: Umbral de porcentaje de valores faltantes para eliminar columnas
# @return: DataFrame con las columnas que tienen un porcentaje de valores faltantes menor o igual al umbral
function eliminar_columnas(df::DataFrame, umbral::Float64)
    println("Eliminando columnas con porcentaje de valores faltantes mayor al umbral $umbral...")
    porcentajes_faltantes = porcentaje_faltantes(df)
    columnas_a_conservar = filter(col -> porcentajes_faltantes[col] <= umbral, names(df))
    println("Columnas conservadas: ", columnas_a_conservar)
    return df[:, columnas_a_conservar]
end

eliminar_columnas (generic function with 1 method)

In [5]:
umbral = 50.0
df_limpio = eliminar_columnas(df, umbral)

Eliminando columnas con porcentaje de valores faltantes mayor al umbral 50.0...
Columnas conservadas: ["Cst_Cnt", "Btl_Cnt", "Sta_ID", "Depth_ID", "Depthm", "T_degC", "Salnty", "O2ml_L", "STheta", "O2Sat", "Oxy_µmol/Kg", "RecInd", "T_prec", "S_prec", "P_qual", "Chlqua", "Phaqua", "PO4q", "SiO3qu", "NO2q", "NO3q", "NH3q", "C14A1q", "C14A2q", "DarkAq", "MeanAq", "R_Depth", "R_TEMP", "R_POTEMP", "R_SALINITY", "R_SIGMA", "R_SVA", "R_DYNHT", "R_O2", "R_O2Sat", "R_PRES"]


Row,Cst_Cnt,Btl_Cnt,Sta_ID,Depth_ID,Depthm,T_degC,Salnty,O2ml_L,STheta,O2Sat,Oxy_µmol/Kg,RecInd,T_prec,S_prec,P_qual,Chlqua,Phaqua,PO4q,SiO3qu,NO2q,NO3q,NH3q,C14A1q,C14A2q,DarkAq,MeanAq,R_Depth,R_TEMP,R_POTEMP,R_SALINITY,R_SIGMA,R_SVA,R_DYNHT,R_O2,R_O2Sat,R_PRES
,Int64,Int64,String15,String,Int64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Int64,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Int64?,Float64,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Int64
1,1,1,054.0 056.0,19-4903CR-HY-060-0930-05400560-0000A-3,0,10.5,33.44,missing,25.649,missing,missing,3,1,2,9,9,9,9,9,9,9,9,9,9,9,9,0.0,10.5,10.5,33.44,25.64,233.0,0.0,missing,missing,0
2,1,2,054.0 056.0,19-4903CR-HY-060-0930-05400560-0008A-3,8,10.46,33.44,missing,25.656,missing,missing,3,2,2,9,9,9,9,9,9,9,9,9,9,9,9,8.0,10.46,10.46,33.44,25.65,232.5,0.01,missing,missing,8
3,1,3,054.0 056.0,19-4903CR-HY-060-0930-05400560-0010A-7,10,10.46,33.437,missing,25.654,missing,missing,7,2,3,9,9,9,9,9,9,9,9,9,9,9,9,10.0,10.46,10.46,33.437,25.65,232.8,0.02,missing,missing,10
4,1,4,054.0 056.0,19-4903CR-HY-060-0930-05400560-0019A-3,19,10.45,33.42,missing,25.643,missing,missing,3,2,2,9,9,9,9,9,9,9,9,9,9,9,9,19.0,10.45,10.45,33.42,25.64,234.1,0.04,missing,missing,19
5,1,5,054.0 056.0,19-4903CR-HY-060-0930-05400560-0020A-7,20,10.45,33.421,missing,25.643,missing,missing,7,2,3,9,9,9,9,9,9,9,9,9,9,9,9,20.0,10.45,10.45,33.421,25.64,234.0,0.04,missing,missing,20
6,1,6,054.0 056.0,19-4903CR-HY-060-0930-05400560-0030A-7,30,10.45,33.431,missing,25.651,missing,missing,7,2,3,9,9,9,9,9,9,9,9,9,9,9,9,30.0,10.45,10.45,33.431,25.65,233.5,0.07,missing,missing,30
7,1,7,054.0 056.0,19-4903CR-HY-060-0930-05400560-0039A-3,39,10.45,33.44,missing,25.658,missing,missing,3,2,2,9,9,9,9,9,9,9,9,9,9,9,9,39.0,10.45,10.45,33.44,25.65,233.0,0.09,missing,missing,39
8,1,8,054.0 056.0,19-4903CR-HY-060-0930-05400560-0050A-7,50,10.24,33.424,missing,25.682,missing,missing,7,2,3,9,9,9,9,9,9,9,9,9,9,9,9,50.0,10.24,10.23,33.424,25.68,231.0,0.11,missing,missing,50
9,1,9,054.0 056.0,19-4903CR-HY-060-0930-05400560-0058A-3,58,10.06,33.42,missing,25.71,missing,missing,3,2,2,9,9,9,9,9,9,9,9,9,9,9,9,58.0,10.06,10.05,33.42,25.71,228.5,0.13,missing,missing,58


## Correlation Matrix

In [16]:
function calcular_correlacion_con_target(df::DataFrame, target::Symbol)
    println("Calculando correlación con la columna target: $target...")

    # Identificar columnas numéricas
    columnas_numericas = filter(c -> eltype(df[!, c]) <: Real, names(df))
    println("Columnas numéricas identificadas: ", columnas_numericas)

    # Verificar si el target está en las columnas numéricas
    if !(string(target) in columnas_numericas)
        error("El target $target no está en las columnas numéricas. Verifica su tipo o nombre.")
    end

    # Subset del DataFrame con las columnas numéricas
    df_numerico = df[:, columnas_numericas]

    # Calcular la matriz de correlación
    matriz_correlacion = pairwise(cor, eachcol(df_numerico))

    # Encontrar índice del target y calcular correlaciones con él
    target_idx = findfirst(columnas_numericas .== string(target))
    correlaciones_target = matriz_correlacion[:, target_idx]

    # Mostrar correlaciones
    println("Correlaciones con $target:")
    for (col, corr) in zip(columnas_numericas, correlaciones_target)
        println("$col: $corr")
    end

    return correlaciones_target, columnas_numericas
end


calcular_correlacion_con_target (generic function with 1 method)

In [17]:
correlaciones, columnas_numericas = calcular_correlacion_con_target(df_limpio, :T_degC)


Calculando correlación con la columna target: T_degC...
Columnas numéricas identificadas: ["Cst_Cnt", "Btl_Cnt", "Depthm", "T_degC", "RecInd", "R_Depth", "R_PRES"]
Correlaciones con T_degC:
Cst_Cnt: 0.08878292114395081
Btl_Cnt: 0.0895806956745245
Depthm: -0.6786277885272518
T_degC: 1.0
RecInd: -0.06606047992918594
R_Depth: -0.6786262820532668
R_PRES: -0.6772888202050212


([0.08878292114395081, 0.0895806956745245, -0.6786277885272518, 1.0, -0.06606047992918594, -0.6786262820532668, -0.6772888202050212], ["Cst_Cnt", "Btl_Cnt", "Depthm", "T_degC", "RecInd", "R_Depth", "R_PRES"])

## Linear Regression

In [20]:
# Crear una fórmula para el modelo de regresión
formula = @formula(T_degC ~ Depthm + R_Depth + R_PRES)

# Ajustar el modelo
modelo = lm(formula, df_limpio)

# Resumen del modelo
println(modelo)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

T_degC ~ 1 + Depthm + R_Depth + R_PRES

Coefficients:
───────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error        t  Pr(>|t|)  Lower 95%  Upper 95%
───────────────────────────────────────────────────────────────────────────
(Intercept)  14.1273    0.00416028  3395.76    <1e-99  14.1192     14.1355
Depthm       -2.44735   0.121974     -20.06    <1e-88  -2.68641    -2.20829
R_Depth       0.973929  0.122048       7.98    <1e-14   0.734719    1.21314
R_PRES        1.44876   0.00255368   567.32    <1e-99   1.44376     1.45377
───────────────────────────────────────────────────────────────────────────


## Evaluación

In [22]:
# Coeficiente de determinación (R²)
r2_value = r2(modelo)
println("R² del modelo: ", r2_value)

# Predicciones
df_limpio.T_degC_pred = predict(modelo, df_limpio)

# Mostrar los coeficientes del modelo
println("Coeficientes del modelo:")
for (nombre, coef) in zip(termnames(modelo), coef(modelo))
    println("$nombre: $coef")
end

R² del modelo: 0.6071503069562575
Coeficientes del modelo:
T_degC: 14.127314257394794
["(Intercept)", "Depthm", "R_Depth", "R_PRES"]: -2.4473499771520206


### Correlación de variables

In [23]:
println(cor(df_limpio.Depthm, df_limpio.R_Depth))
println(cor(df_limpio.Depthm, df_limpio.R_PRES))
println(cor(df_limpio.R_Depth, df_limpio.R_PRES))

0.9999999972818153
0.9999939222785432
0.9999939296557131


## Opciones de variables

In [24]:
formula1 = @formula(T_degC ~ Depthm + R_Depth)
modelo1 = lm(formula1, df_limpio)
println("R² (sin R_PRES): ", r2(modelo1))

R² (sin R_PRES): 0.46095207144967487


In [25]:
formula2 = @formula(T_degC ~ Depthm + R_Depth + Cst_Cnt + Btl_Cnt)
modelo2 = lm(formula2, df_limpio)
println("R² (con Cst_Cnt y Btl_Cnt): ", r2(modelo2))

R² (con Cst_Cnt y Btl_Cnt): 0.46250680859778814


In [26]:
formula3 = @formula(T_degC ~ Depthm + R_Depth + R_PRES + RecInd)
modelo3 = lm(formula3, df_limpio)
println("R² (con RecInd): ", r2(modelo3))

R² (con RecInd): 0.6071679162519756


In [27]:
formula4 = @formula(T_degC ~ Depthm + R_Depth + R_PRES + Cst_Cnt + Btl_Cnt + RecInd)
modelo4 = lm(formula4, df_limpio)
println("R² (todas las variables): ", r2(modelo4))

R² (todas las variables): 0.6098434894989748
